# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
!pip install geoviews
# api_keys.py

geoapify_key = "2c44fef549a1435cad30c64d24d5411a"

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,constantia,44.1833,28.6500,274.65,77,2,6.05,RO,1701304773
1,1,port-aux-francais,-49.3500,70.2167,275.40,97,100,21.47,TF,1701304773
2,2,bethel,41.3712,-73.4140,273.45,55,0,3.09,US,1701304603
3,3,fale old settlement,-9.3852,-171.2468,302.14,74,87,5.55,TK,1701304773
4,4,kerikeri,-35.2268,173.9474,295.32,64,100,0.89,NZ,1701304774


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
import geoviews as gv
import holoviews as hv
import pandas as pd

# Load the data
city_data_df = pd.read_csv('output_data/cities.csv')

# Create a GeoViews Points element
points = gv.Points(city_data_df, ['Lng', 'Lat'], ['Humidity'])

# Configure the plot
plot = points.opts(size='Humidity', color='blue', cmap='viridis', tools=['hover'], width=800, height=500)

# Display the map
plot


:Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities based on weather conditions
ideal_conditions = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_conditions = ideal_conditions.dropna()

# Display sample data
ideal_conditions.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,constantia,RO,44.1833,28.6500,77,
1,port-aux-francais,TF,-49.3500,70.2167,97,
2,bethel,US,41.3712,-73.4140,55,
3,fale old settlement,TK,-9.3852,-171.2468,74,
4,kerikeri,NZ,-35.2268,173.9474,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000  # Assuming a search radius of 10,000 meters
params = {
    "radius": radius,
    # Additional parameters based on the Geoapify API documentation
    # For example: "type": "hotel" to specifically search for hotels
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lat},{lng}"
    params["bias"] = f"point:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
constantia - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
fale old settlement - nearest hotel: No hotel found
kerikeri - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
puerto san jose - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
caconda - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: No hotel found
enewetak - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
formosa - nearest hotel: No hotel found
whitehorse - nearest hotel: No hotel found
iqaluit - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
tamboril - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
karratha

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
import geoviews as gv
import holoviews as hv
import pandas as pd

# Load the data
city_data_df = pd.read_csv('output_data/cities.csv')

# Define the Points element for the city locations
points = gv.Points(city_data_df, ['Lng', 'Lat'], ['City', 'Country', 'Humidity'])

# Combine Points and hover information
plot = gv.tile_sources.OSM * points.opts(tools=['hover'], color='blue', cmap='viridis', size='Humidity')

# Display the map
plot.opts(width=800, height=500)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity)